In [129]:
import numpy as np
import scipy
from scipy.sparse import diags
from sys import getsizeof
from numpy.linalg import multi_dot as md
from tqdm.notebook import tqdm

In [130]:
class ImplicitNaiveALS(object):
    def __init__(self, factors=10, iterations=100, lambda_=4, alpha=40, seed=10):
        self.iterations = iterations
        self.lambda_ = lambda_
        self.alpha = alpha
        self.factors = factors
        self.Yi = None
        self.Xu = None
        self.seed = seed
    
    def fit(self, matrix):
        np.random.seed(self.seed)
        Pui = (matrix > 0).astype('int')
        Cui = 1 + self.alpha * matrix
        Xu = 0.02 * np.random.normal(0, size=(matrix.shape[0], self.factors))
        Yi = 0.02 * np.random.normal(0, size=(matrix.shape[1], self.factors))
        
        for _iter in tqdm(range(self.iterations)):
            
            for user in range(matrix.shape[0]):
                a = md([Yi.T, np.diag(Cui[user]), Yi]) + self.lambda_ * np.eye(self.factors)
                b = md([Yi.T, np.diag(Cui[user]), Pui[user]])
                Xu[user] = np.linalg.solve(a, b)
            
            for item in range(matrix.shape[1]):
                a = md([Xu.T, np.diag(Cui[:, item]), Xu]) + self.lambda_ * np.eye(self.factors)
                b = md([Xu.T, np.diag(Cui[:, item]), Pui[:, item]])
                Yi[item] = np.linalg.solve(a, b)
                
        self.Xu = Xu
        self.Yi = Yi        
    
    
    def predict(self, user_id, top=10):
        recommendations = np.dot(self.Xu[user_id], self.Yi.T).argsort(axis=1)[:, -top:][:, ::-1]
        return recommendations

In [131]:
a = ImplicitNaiveALS(factors=20, iterations=100)

In [132]:
temp = 10 * np.eye(100) + 0.0001 * np.random.normal(0, size=(100,100))

In [133]:
a.fit(temp)

In [134]:
a.predict(user_id=[1,2,3])

array([[ 1, 26, 25, 14, 38, 54, 20, 13, 91, 70],
       [ 2, 59, 61, 15, 60, 90, 51, 92, 87, 28],
       [ 3, 41, 17, 68, 54, 70, 77, 26, 47, 55]])